# Gordetako soluzioen azterketa eta erdiguneko metodo inplizituaren eraginkortasuna

<ul id="top">
<li><a href="#Pakete-eta-funtzioak-kargatu"> 
    Pakete eta funtzioak kargatu</a></li>
<li><a href="#Gordetako-soluzio-periodikoak-irakurri">
    Gordetako soluzio periodikoak irakurri</a></li>
<li><a href="#Une-kolinealak-aztertu">
    Une kolinealak aztertu</a></li>
<li><a href="#Fast-Fourier-Transform">
    Fast Fourier Transform</a></li>
<li><a href="#Gordetako-soluzio-periodiko-zaharrak-irakurri-(3-fasetan-antolatutako-bilaketa)">
    Gordetako soluzio periodiko zaharrak irakurri (3 fasetan antolatutako bilaketa)</a></li>
<li><a href="#Erdiguneko-Metodo-Inplizituan-estrapolazioaren-eraginkortasuna-aztertu">
    Erdiguneko Metodo Inplizituan estrapolazioaren eraginkortasuna aztertu</a></li> 
</ul>  

## Pakete eta funtzioak kargatu

In [ ]:
using LinearAlgebra
using Plots 
using Optim
using BenchmarkTools
using Combinatorics
using CSV
using DataFrames
using ForwardDiff
using FFTW

In [ ]:
include("./src/EMI.jl"); include("./src/TBP.jl"); include("./src/utils.jl"); include("./src/Loss.jl");

## Gordetako soluzio periodikoak irakurri

In [ ]:
#filepath = "./results/older/TBP_automatizatuta.csv"
#filepath = "./results/older/TBP_automatizatuta_filtered.csv"
#filepath = "./results/TBP_automatizatuta.csv"
#filepath = "./results/TBP_automatizatuta_filtered.csv"
filepath = "./results/TBP_automatizatuta_filtered_filtered.csv"

In [ ]:
z0_list, Zopt_list, n_list, sigma_list = CSVIrakurri(filepath);
i = 0;

In [ ]:
i+=1
println("i = ", i)
T = Zopt_list[i][5]
n = n_list[i]
dtau = T/n
F3 = FLoss3(sigma_list[i], n)
println("h: ", Zopt_list[i][6])
println("T: ", T)
println("dtau: ", dtau)
println("z0: ", z0_list[i])
println("Zopt: ", Zopt_list[i])
println("Helburu funtzioa: ", F3(Zopt_list[i]));

In [ ]:
periodoa = 2T
println("Periodoa: ", periodoa)
tt, UU, uu = Irudikatu(Zopt_list[i], 0., periodoa, dtau, Zopt_list[i][6]);
#Kodea lortzeko urratsak
g_col_list = [g_col(u) for u in uu] 
code_middle, code_far, ind_kolinear_list = getCode(UU, g_col_list)
println("Kodea_middle: ", join(code_middle,""))
println("Kodea_far: ", join(code_far,""))

## Une kolinealak aztertu

In [ ]:
j = 0;

In [ ]:
j+=1
Irudikatu(Zopt_list[i], 0., tt[ind_kolinear_list[j]], dtau, Zopt_list[i][6]); 


## Fast Fourier Transform

In [ ]:
II = I_fcn.(UU[1:end-1])
hII = rfft(II)/length(II)
loghII = log10.(abs.(hII))
scatter(loghII)

### Egoera bat irudikatu eta gordetzeko kodea

### Ibilbidea irudikatu eta gordetzeko kodea

## Gordetako soluzio periodiko zaharrak irakurri

In [ ]:
filepath = "./results/older/TBP_periodikoak.csv"
list1, list2, list3, list4 = csvIrakurri(filepath);
k = 0;

In [ ]:
k+=1
println("k = ", k)
z0 = list4[k]
println("z0: ", z0)
F2 = FLoss2(list3[k][1], list3[k][2], list3[k][3])
println("F2: ", F2(list2[k]))
println(list3[k][1])

In [ ]:
tt, UU, uu = Irudikatu(list2[k], 0., 2list2[k][5], list2[k][5]/list3[k][2], list3[k][3]);
g_col_list = [g_col(u) for u in uu]
code_middle, code_far, ind_kolinear_list = getCode(UU, g_col_list)
println("Kodea_far: ", code_far)
println("Kodea_middle: ", code_middle)

## Erdiguneko Metodo Inplizituan estrapolazioaren eraginkortasuna aztertu

In [ ]:
iterazio_kopurua = 50

dtaus = [1/32,1/64, 1/128, 1/256, 1/512]

times32 = [Float64[] for i in 1:6]
iter32 = [Float64[] for i in 1:6]

times64 = [Float64[] for i in 1:6]
iter64 = [Float64[] for i in 1:6]

times128 = [Float64[] for i in 1:6]
iter128 = [Float64[] for i in 1:6]

times256 = [Float64[] for i in 1:6]
iter256 = [Float64[] for i in 1:6]

times512 = [Float64[] for i in 1:6]
iter512 = [Float64[] for i in 1:6]

times = [times32, times64, times128, times256, times512]
iter = [iter32, iter64, iter128, iter256, iter512]

extrakop = [0,2,4,6,8,10]

for k in 1:iterazio_kopurua
    #Ausazko hasierako egoera lortu
    z0 = pi*rand(4)
    u0 = HasierakoEgoera(z0)
    U0 = LCinvFcn(u0)
    h = ThreeBodyRelEnergy(u0)

    for i in 1:length(dtaus) #delta tau desberdinetarako
        for j in 1:6 #estrapolazio kopuru bakoitzeko
            if j==1 #estrapolaziorik ez
                elapsed_time = @elapsed begin
                    _, lag = EMIiter0(U0, 0.,20.,dtaus[i], [h])
                end
            else #estrapolazioarekin
                elapsed_time = @elapsed begin
                    _, lag = EMIiter(U0, 0.,20.,dtaus[i],[h], extrakop[j])
                end
            end
            push!(iter[i][j], lag)
            push!(times[i][j], elapsed_time)
        end
    end 
end

#Bistaratu emaitzak
for i in 1:5
    println("_________________________________________")
    println("dtau: ", dtaus[i])
    println("_________________________________________")
    for j in 1:6
        println("Nabla ordena: ", extrakop[j])
        println("Iterazio batezbestekoa: ", sum(iter[i][j])/length(iter[i][j]))
        println("Denbora batezbestekoa: ", sum(times[i][j])/length(times[i][j]))
    end
end